how to convert anki package to study sauce?


how to sign a file url for uploading to google storage?


In [ ]:
const {Storage} = require('@google-cloud/storage');
const storage = new Storage();

/**
 * HTTP function that generates a signed URL
 * The signed URL can be used to upload files to Google Cloud Storage (GCS)
 *
 * @param {Object} req Cloud Function request context.
 * @param {Object} res Cloud Function response context.
 */
exports.getSignedUrl = (req, res) => {
  if (req.method !== 'POST') {
    // Return a "method not allowed" error
    return res.status(405).end();
  }
  // TODO(developer) check that the user is authorized to upload

  // Get a reference to the destination file in GCS
  const file = storage.bucket(req.body.bucket).file(req.body.filename);

  // Create a temporary upload URL
  const expiresAtMs = Date.now() + 300000; // Link expires in 5 minutes
  const config = {
    action: 'write',
    expires: expiresAtMs,
    contentType: req.body.contentType,
  };

  file.getSignedUrl(config, (err, url) => {
    if (err) {
      console.error(err);
      res.status(500).end();
      return;
    }
    res.send(url);
  });
};



create a copy of study sauce template?


In [ ]:
var uuid = require('uuid/v1');
var importer = require('../Core');
var getSheet = importer.import('get sheet identifier');
var copyFile = importer.import('copy a file on google drive');
var listDrive = importer.import('list google drive files');
var insertPermission = importer.import('insert google drive permissions');

function copyStudy(email) {
    var fileId;
    return listDrive()
        .then(r => fileId = r.filter(f => f.name === 'Study sauce template')[0].id)
        .then(() => copyFile(fileId, 'Study sauce ' + uuid().substr(0, 5)))
        .then(id => insertPermission(id, email))
        .then(() => fileId)
}

async function handler(req, res) {
    var file;
    var email = (req.body || {})['email'] || (req.form || {})['email'] || (req.query || {})['email'];
    res.set('Access-Control-Allow-Origin', '*');
    return await copyStudy(email)
        // add sheet to signup script with email
        .then(fileId => file = fileId)
        .then(() => getSheet(file, email))
        .then(r => res.status(200).send(file))
        .catch(e => res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key] + '';
            return alt;
        }, {})));
}

module.exports = copyStudy;
module.exports.handler = handler



render study sauce cards page?


In [ ]:
var uuid = require('uuid/v1');
var {Readable} = require('stream');
var getTemplates = importer.import('templates google sheet');
var wrapTemplate = importer.import('output google sheet template');
var getTemplateProperties = importer.import('google sheet template properties');
var collectTemplateResources = importer.import('collect google sheet resources');
var streamToGoogle = importer.import('upload files google cloud');

var config = 'https://docs.google.com/spreadsheets/d/1EGwxT6InTXuvpLujnwKV3asEFDZhhZk2LdosjW2Tz_M/edit#gid=851799338'
var bucket = 'studysauce.sheet-to-web.com';
var domains = [
    'https://localhost:8080',
    'https://studysauce.sheet-to-web.com'
]

function createStream(newPage) {
    var stream = new Readable();
    stream.push(newPage);
    stream.push(null);
    return Promise.resolve(stream);
}

function parseCards(str) {
    debugger;
    return str.split('\n').map(line => {
        var values = line.split(',');
        return {
            type: values[0],
            prompt: values[1],
            answer: values[2],
            'possible-1': values[2],
            'possible-2': values[3],
            'possible-3': values[4],
            'possible-4': values[5]
        }
    })
}

// TODO: make this generic for use with user related packs and state changes?
function renderCards(cards) {
    if(typeof cards === 'string') {
        cards = parseCards(cards)
    }
    // TODO: create a document and render cards from that
    var docId = config.replace(/https:\/\/docs.google.com\/spreadsheets\/d\/|\/edit.*$|\/copy.*$/ig, '');
    var name = uuid().substr(0, 5) + (new Date()).getTime();
    var properties = {}, templates;
    
    return getTemplates(docId)
        .then(t => {
            templates = t;
            templates['cards'].data.rows = properties['cards-data'];
            templates['cards-cards'] = {template: {rows: [[`{{> cards/cards-cards-link}}`]]}}
            return getTemplateProperties('app', properties, templates);
        })
        .then(() => getTemplateProperties('demo', properties, templates))
        .then(() => getTemplateProperties('cards-cards', properties, templates))
        .then(() => wrapTemplate('cards/' + name, properties['cards-cards'], Object.assign(properties, {
            'cards-cards-link': '/cards/' + name,
            'cards-data': (cards || parseCards(properties['demo-text']))
                .map(c => Object.assign(c, {cards: '/cards/' + name}))
        })))
        .then(page => createStream(page))
        .then(stream => streamToGoogle('cards/' + name + '.html', bucket, stream, {
            contentType: 'text/html; charset=utf-8'
        }))
        .then(resources => {
            console.log(resources);
            return '/cards/' + name + '.html';
        })
}

// TODO: generic RPC handler for exporting with any function?
async function handler(req, res) {
    var file;
    var cards = (req.body || {})['cards'] || (req.form || {})['cards'] || (req.query || {})['cards'];
    res.set('Access-Control-Allow-Headers', 'content-type');
    res.set('Access-Control-Allow-Origin', domains.filter(d => d.includes(req.get('origin')))[0] || '*');
    res.type('text/plain; charset=utf-8');
    if (req.method == 'OPTIONS') {
        return res.status(204).send('');
    }
    return await renderCards(cards)
        // add sheet to signup script with email
        .then(r => res.status(200).send(r))
        .catch(e => res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key] + '';
            return alt;
        }, {})));
}

module.exports = renderCards;
module.exports.handler = handler;



authorize app to read profile info?


In [ ]:
var path = require('path');
var fs = require('fs');
var util = require('util');
var session = require('express-session')
var uuid = require('uuid/v1');
var {OAuth2Client} = require('google-auth-library');

var importer = require('../Core');
var getDataSheet = importer.import('get google data sheet');
var updateRow = importer.import('update row in google sheets');

var redirect = 'http://localhost:8010/sheet-to-web/us-central1/authorizeSauce';
var SCOPES = [
    'https://www.googleapis.com/auth/userinfo.profile',
    'https://www.googleapis.com/auth/userinfo.email'
];
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var doc = 'https://docs.google.com/spreadsheets/d/1EGwxT6InTXuvpLujnwKV3asEFDZhhZk2LdosjW2Tz_M/edit#gid=0'
var credentials;
if(fs.existsSync('./client_secret.json')) {
    credentials = require(path.resolve('./client_secret.json'));
} else {
    credentials = require(path.join(PROFILE_PATH, '.credentials/client_secret.json'));
}

function safeName(name) {
    return name.replace(/[^a-z0-9\-]/ig, '-').substr(0, 40).toLowerCase();
}

function extractProfile(oauth2Client) {
    return util.promisify(oauth2Client.request.bind(oauth2Client))({
            url: 'https://www.googleapis.com/oauth2/v3/userinfo?alt=json'
        })
        .then(r => {
            var profile = r.data;
            console.log(profile);
            var rowData = {
                link: safeName(profile.name) + '-' + uuid().substr(0, 5),
                first: profile.given_name,
                last: profile.family_name,
                email: profile.email,
                token: JSON.stringify(oauth2Client.credentials),
                image: profile.picture || profile.photos[0].value,
                google: profile.id,
                username: profile.name
            }
            return updateRow(doc, u => u.email === profile.email, rowData, 'User data')
        })
}

function receiveCode(req, res) {
    var oauth2Client = new OAuth2Client(
        credentials.installed.client_id,
        credentials.installed.client_secret,
        redirect),
        email = ((req || {}).session || {}).email

    var token;
    return util.promisify(oauth2Client.getToken.bind(oauth2Client))(req.query['code'])
        .then(t => oauth2Client.setCredentials(token = t))
        .then(() => extractProfile(oauth2Client))
        .then(profile => {
            if(typeof req.session === 'undefined') {
                req.session = {};
            }
            req.session.email = profile.email;
            return profile
        })
}

function authorizeProfile(req, res, scopes = SCOPES) {
    var oauth2Client = new OAuth2Client(
        credentials.installed.client_id,
        credentials.installed.client_secret,
        redirect),
        email = ((req || {}).session || {}).email

    return getDataSheet(doc, 'User data')
        .then(users => users.filter(u => u.email === email)[0])
        .then(user => {
            var token = JSON.parse((user || {}).token || '""');
            if(token) {
                oauth2Client.setCredentials(token)
                return oauth2Client.getAccessToken();
            }
            throw new Error('invalid_token');
        })
        .then(() => extractProfile(oauth2Client))
        .catch(up => {
            if(up.message == 'invalid_token') {
                var authUrl = oauth2Client.generateAuthUrl({
                    access_type: 'offline',
                    scope: scopes
                });
                res.redirect(authUrl);
                return authUrl;
            } else {
                throw up;
            }
        })
}

async function handler(req, res) {
    return Promise.resolve(true) /*await new Promise(resolve => session({
            secret: 'keyboard cat',
            resave: false,
            saveUninitialized: true,
            cookie: {}
        }, resolve))*/
        .then(r => console.log(r))
        .then(() => (req.query['code'] ? receiveCode : authorizeProfile)(req, res))
        // add sheet to signup script with email
        .then(r => res.status(200).send(r))
        .catch(e => res.status(500).send(Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key] + '';
            return alt;
        }, {})))
}

module.exports = authorizeProfile;
module.exports.handler = handler;



package.json?


In [ ]:
{
    "name": "SheetToWeb",
    "description": "Marketing site functions",
    "license": "UNLICENSED",
    "scripts": {
    },
    "engines": {
        "node": ">= 8",
        "npm": ">= 4"
    },
    "repository": {
        "type": "git",
        "url": "git+https://github.com/megamindbrian/jupytangular.git"
    },
    "dependencies": {
        "@google-cloud/compute": "^0.12.0",
        "@google-cloud/storage": "^2.5.0",
        "googleapis": "^39.2.0",
        "jsdom": "^14.0.0",
        "mustache": "^3.0.1",
        "remarkable": "^1.7.1"
    }
}
